In [19]:
from pathlib import Path
import pandas as pd
import json


## Evaluation of different allocation approaches

In [20]:
directory = Path("evaluation")

In [21]:
# Get list of all considered files
files = []
for filepath in directory.rglob("*"):  # Matches all files and directories
    if filepath.is_file():
        files.append(filepath)
files

[PosixPath('evaluation/paper_process_offer_resources_heterogen/all_instance_cp.json'),
 PosixPath('evaluation/paper_process_offer_resources_heterogen/single_instance_cp_warm.json'),
 PosixPath('evaluation/paper_process_offer_resources_heterogen/single_instance_cp.json'),
 PosixPath('evaluation/paper_process_offer_resources_heterogen/heuristic.json'),
 PosixPath('evaluation/paper_process_short_invalids/single_instance_cp_warm.json'),
 PosixPath('evaluation/paper_process_short_invalids/single_instance_cp.json'),
 PosixPath('evaluation/paper_process_short_invalids/heuristic.json'),
 PosixPath('evaluation/paper_process_offer_resources_close_maxima/all_instance_cp.json'),
 PosixPath('evaluation/paper_process_offer_resources_close_maxima/single_instance_cp_warm.json'),
 PosixPath('evaluation/paper_process_offer_resources_close_maxima/single_instance_cp.json'),
 PosixPath('evaluation/paper_process_offer_resources_close_maxima/heuristic.json')]

In [ ]:
data = {}
for file in files:
    print(file)
    with open(file) as f:
        object = json.load(f)
    data[str(file)] = object["solution"]


evaluation/paper_process_offer_resources_heterogen/all_instance_cp.json
evaluation/paper_process_offer_resources_heterogen/single_instance_cp_warm.json
evaluation/paper_process_offer_resources_heterogen/single_instance_cp.json
evaluation/paper_process_offer_resources_heterogen/heuristic.json


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [13]:
df = pd.DataFrame.from_dict(data, orient="index")
df.sort_values(by="objective")

,objective,optimality gap,computing time,solver status,branches,propagations,total interval length
evaluation/paper_process_short_invalids/single_instance_cp_warm.json,41.0,N/A,92.440000,Optimal,448155.0,N/A,53.0
evaluation/paper_process_short_invalids/single_instance_cp.json,41.0,N/A,174.830000,Optimal,413338.0,N/A,53.0
evaluation/paper_process_short_invalids/heuristic.json,42.0,NaN,5.600772,NaN,NaN,NaN,54.0
evaluation/paper_process_offer_resources_close_maxima/all_instance_cp.json,200.0,N/A,1469.750000,Feasible,208959464.0,N/A,549.0
evaluation/paper_process_offer_resources_close_maxima/heuristic.json,211.0,NaN,0.392110,NaN,NaN,NaN,582.0
evaluation/paper_process_offer_resources_close_maxima/single_instance_cp.json,237.0,N/A,4.430000,Optimal,625889.0,N/A,550.0
evaluation/paper_process_offer_resources_close_maxima/single_instance_cp_warm.json,237.0,N/A,4.060000,Optimal,635699.0,N/A,535.0
evaluation/paper_process_offer_resources_heterogen/all_instance_cp.json,680.0,N/A,68.720000,Optimal,17043705.0,N/A,1797.0
evaluation/paper_process_offer_resources_heterogen/heuristic.json,702.0,NaN,0.344974,NaN,NaN,NaN,1629.0
evaluation/paper_process_offer_resources_heterogen/single_instance_cp.json,775.0,N/A,0.560000,Optimal,47985.0,N/A,1791.0
